## Installation

To run this notebook, install Panel:

```bash
pip install panel pandas numpy
```

Or if using conda:

```bash
conda install -c conda-forge panel pandas numpy
```


# Panel Widget Mockup for DataFrame Building

This notebook demonstrates Panel's capabilities for building interactive DataFrame interfaces, similar to the ipywidgets implementation but using HoloViz Panel.

## Features:

- Global parameter controls
- Dynamic table with add/remove rows and columns
- Interactive data editing
- Save/Load functionality mockup
- Modern responsive layout


In [ ]:
# Import required libraries
from io import StringIO

import numpy as np
import pandas as pd
import panel as pn

# Initialize Panel extension
pn.extension("tabulator", sizing_mode="stretch_width")

## 1. Basic Panel DataFrame Widget

First, let's create a simple editable table using Panel's Tabulator widget:


In [ ]:
# Create a simple DataFrame
simple_df = pd.DataFrame(
    {
        "Sample Theta": [1, 5, 10, 15, 20],
        "Higher Order Suppressor": [10, 9, 8.5, 7.5, 7.5],
        "Exposure": [0.001, 0.001, 0.001, 0.1, 0.5],
    }
)

# Create an editable Tabulator widget
simple_table = pn.widgets.Tabulator(
    simple_df, width=600, height=300, layout="fit_columns", name="Basic Table"
)

simple_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'637764be-cc3d-4925-9b92-101314acbbc3': {'version…

## 2. Complete Interactive Application

Now let's build a complete application similar to your ipywidgets implementation with:

- Global parameters section
- Dynamic table with add/remove controls
- Multiple measurement tabs
- Save/Load functionality


In [ ]:
class PanelDataFrameBuilder:
    """
    A Panel-based interactive DataFrame builder with global parameters
    and dynamic table editing capabilities.
    """

    def __init__(self):
        # Initialize data storage
        self.current_df = pd.DataFrame(
            {
                "Step": [1, 2, 3, 4, 5],
                "Sample Theta": [1, 5, 10, 15, 20],
                "Higher Order Suppressor": [10, 9, 8.5, 7.5, 7.5],
                "Exposure": [0.001, 0.001, 0.001, 0.1, 0.5],
            }
        )

        # Global Parameters Section
        self.sample_name = pn.widgets.TextInput(
            name="Sample Name", value="Sample_01", width=300
        )

        self.energy_start = pn.widgets.FloatInput(
            name="Energy Start [eV]", value=250.0, start=0, end=1000, width=300
        )

        self.energy_stop = pn.widgets.FloatInput(
            name="Energy Stop [eV]", value=280.0, start=0, end=1000, width=300
        )

        self.energy_step = pn.widgets.FloatInput(
            name="Energy Step [eV]", value=0.5, start=0, end=100, width=300
        )

        self.theta_final = pn.widgets.FloatInput(
            name="Final Theta [deg]", value=60.0, start=0, end=90, width=300
        )

        self.x_position = pn.widgets.FloatInput(
            name="X Position [mm]", value=0.0, width=300
        )

        self.y_position = pn.widgets.FloatInput(
            name="Y Position [mm]", value=0.0, width=300
        )

        # Create editable table
        self.table = pn.widgets.Tabulator(
            self.current_df,
            width=700,
            height=400,
            layout="fit_columns",
            editors={
                "Step": None,  # Read-only
                "Sample Theta": {"type": "number", "min": 0, "max": 90, "step": 0.1},
                "Higher Order Suppressor": {
                    "type": "number",
                    "min": 0,
                    "max": 20,
                    "step": 0.1,
                },
                "Exposure": {"type": "number", "min": 0.001, "max": 10, "step": 0.001},
            },
            buttons={"Delete": "<i class='fa fa-trash'></i>"},
            name="Measurement Table",
        )

        # Control buttons
        self.add_row_button = pn.widgets.Button(
            name="Add Row", button_type="primary", width=150
        )
        self.add_row_button.on_click(self.add_row)

        self.remove_row_button = pn.widgets.Button(
            name="Remove Last Row", button_type="danger", width=150
        )
        self.remove_row_button.on_click(self.remove_row)

        self.add_column_button = pn.widgets.Button(
            name="Add Column", button_type="success", width=150
        )
        self.add_column_button.on_click(self.add_column)

        self.reset_button = pn.widgets.Button(
            name="Reset Table", button_type="warning", width=150
        )
        self.reset_button.on_click(self.reset_table)

        # Column name input for adding new columns
        self.new_column_name = pn.widgets.TextInput(
            name="New Column Name", placeholder="Enter column name", width=200
        )

        self.new_column_default = pn.widgets.FloatInput(
            name="Default Value", value=0.0, width=200
        )

        # Status indicator
        self.status = pn.pane.Markdown("**Status:** Ready", width=700)

        # Save/Load functionality
        self.save_name = pn.widgets.TextInput(
            name="Filename", value="measurement_data", width=300
        )

        self.save_button = pn.widgets.Button(
            name="Save to CSV", button_type="success", width=150
        )
        self.save_button.on_click(self.save_data)

        self.export_button = pn.widgets.Button(
            name="Export Summary", button_type="primary", width=150
        )
        self.export_button.on_click(self.export_summary)

    def add_row(self, event):
        """Add a new row to the table"""
        current_data = self.table.value
        new_step = len(current_data) + 1

        # Create new row with default values
        new_row = {
            "Step": new_step,
            "Sample Theta": 0.0,
            "Higher Order Suppressor": 10.0,
            "Exposure": 0.001,
        }

        # Add any additional columns that exist
        for col in current_data.columns:
            if col not in new_row:
                new_row[col] = 0.0

        # Append new row
        new_df = pd.concat([current_data, pd.DataFrame([new_row])], ignore_index=True)
        self.table.value = new_df
        self.status.object = f"**Status:** Added row {new_step}"

    def remove_row(self, event):
        """Remove the last row from the table"""
        current_data = self.table.value
        if len(current_data) > 1:
            self.table.value = current_data.iloc[:-1].reset_index(drop=True)
            self.status.object = f"**Status:** Removed last row"
        else:
            self.status.object = (
                f"**Status:** Cannot remove - at least one row required"
            )

    def add_column(self, event):
        """Add a new column to the table"""
        current_data = self.table.value
        col_name = self.new_column_name.value
        default_val = self.new_column_default.value

        if col_name and col_name not in current_data.columns:
            current_data[col_name] = default_val
            self.table.value = current_data
            self.status.object = f'**Status:** Added column "{col_name}"'
            self.new_column_name.value = ""  # Clear input
        elif col_name in current_data.columns:
            self.status.object = f'**Status:** Column "{col_name}" already exists'
        else:
            self.status.object = f"**Status:** Please enter a column name"

    def reset_table(self, event):
        """Reset table to initial state"""
        self.current_df = pd.DataFrame(
            {
                "Step": [1, 2, 3, 4, 5],
                "Sample Theta": [1, 5, 10, 15, 20],
                "Higher Order Suppressor": [10, 9, 8.5, 7.5, 7.5],
                "Exposure": [0.001, 0.001, 0.001, 0.1, 0.5],
            }
        )
        self.table.value = self.current_df
        self.status.object = "**Status:** Table reset to initial state"

    def save_data(self, event):
        """Save current data to CSV"""
        filename = self.save_name.value
        if not filename.endswith(".csv"):
            filename += ".csv"

        current_data = self.table.value
        current_data.to_csv(filename, index=False)
        self.status.object = f"**Status:** Data saved to {filename}"

    def export_summary(self, event):
        """Export a summary including global parameters and table data"""
        summary = {
            "Sample Name": self.sample_name.value,
            "Energy Start [eV]": self.energy_start.value,
            "Energy Stop [eV]": self.energy_stop.value,
            "Energy Step [eV]": self.energy_step.value,
            "Final Theta [deg]": self.theta_final.value,
            "X Position [mm]": self.x_position.value,
            "Y Position [mm]": self.y_position.value,
        }

        # Create summary text
        summary_text = "# Measurement Summary\n\n## Global Parameters\n"
        for key, value in summary.items():
            summary_text += f"- **{key}**: {value}\n"

        summary_text += f"\n## Table Data\n{len(self.table.value)} rows, {len(self.table.value.columns)} columns\n"

        filename = f"{self.save_name.value}_summary.txt"
        with open(filename, "w") as f:
            f.write(summary_text)
            f.write("\n\n## Data Table\n")
            f.write(self.table.value.to_string())

        self.status.object = f"**Status:** Summary exported to {filename}"

    def build_layout(self):
        """Build the complete Panel layout"""

        # Global parameters section
        global_params = pn.Column(
            pn.pane.Markdown(
                "### Global Parameters",
                styles={"background": "#f0f0f0", "padding": "10px"},
            ),
            pn.Row(self.sample_name, self.energy_start),
            pn.Row(self.energy_stop, self.energy_step),
            pn.Row(self.theta_final, self.x_position),
            self.y_position,
            styles={"background": "#f8f8f8", "padding": "15px", "border-radius": "5px"},
            width=750,
        )

        # Table control section
        table_controls = pn.Column(
            pn.pane.Markdown(
                "### Table Controls",
                styles={"background": "#f0f0f0", "padding": "10px"},
            ),
            pn.Row(self.add_row_button, self.remove_row_button, self.reset_button),
            pn.pane.Markdown("#### Add New Column"),
            pn.Row(
                self.new_column_name, self.new_column_default, self.add_column_button
            ),
            styles={"background": "#f8f8f8", "padding": "15px", "border-radius": "5px"},
            width=750,
        )

        # Data table section
        table_section = pn.Column(
            pn.pane.Markdown(
                "### Measurement Data Table",
                styles={"background": "#f0f0f0", "padding": "10px"},
            ),
            self.table,
            self.status,
            styles={"background": "#f8f8f8", "padding": "15px", "border-radius": "5px"},
            width=750,
        )

        # Save/Export section
        save_section = pn.Column(
            pn.pane.Markdown(
                "### Save & Export", styles={"background": "#f0f0f0", "padding": "10px"}
            ),
            pn.Row(self.save_name, self.save_button, self.export_button),
            styles={"background": "#f8f8f8", "padding": "15px", "border-radius": "5px"},
            width=750,
        )

        # Main application layout
        app = pn.Column(
            pn.pane.Markdown(
                "# Interactive DataFrame Builder",
                styles={
                    "background": "#4a86e8",
                    "color": "white",
                    "padding": "20px",
                    "font-size": "24px",
                },
            ),
            pn.pane.Markdown(
                "*Build and customize measurement tables with global parameters*",
                styles={"font-style": "italic", "padding": "10px"},
            ),
            global_params,
            pn.layout.Divider(),
            table_controls,
            pn.layout.Divider(),
            table_section,
            pn.layout.Divider(),
            save_section,
            width=800,
            styles={"background": "white"},
        )

        return app


# Create instance and build the interface
builder = PanelDataFrameBuilder()
app = builder.build_layout()
app

BokehModel(combine_events=True, render_bundle={'docs_json': {'90653357-ffe2-444e-b3db-2862b73ea927': {'version…

## 3. Advanced Features - Tabs with Multiple Experiments

Let's create a more complex example with tabs for multiple experiments/samples:


In [ ]:
class MultiExperimentBuilder:
    """
    Advanced builder with multiple experiment tabs, similar to ipywidgets Tab structure
    """

    def __init__(self):
        self.experiments = {}
        self.experiment_count = 0

        # Global script name
        self.script_name = pn.widgets.TextInput(
            name="Script Name", value="measurement_script", width=300
        )

        # Tab container for experiments
        self.tabs = pn.Tabs(width=850, height=600)

        # Control buttons for managing tabs
        self.add_experiment_button = pn.widgets.Button(
            name="Add Experiment", button_type="primary", width=150
        )
        self.add_experiment_button.on_click(self.add_experiment)

        self.remove_experiment_button = pn.widgets.Button(
            name="Remove Current", button_type="danger", width=150
        )
        self.remove_experiment_button.on_click(self.remove_experiment)

        self.duplicate_experiment_button = pn.widgets.Button(
            name="Duplicate Current", button_type="success", width=150
        )
        self.duplicate_experiment_button.on_click(self.duplicate_experiment)

        # Add initial experiment
        self.add_experiment(None)

    def create_experiment_panel(self, exp_id, template_data=None):
        """Create a single experiment panel"""

        # Use template data if provided, otherwise use defaults
        if template_data is None:
            df = pd.DataFrame(
                {
                    "Step": [1, 2, 3],
                    "Sample Theta": [1, 5, 10],
                    "Higher Order Suppressor": [10, 9, 8.5],
                    "Exposure": [0.001, 0.001, 0.001],
                }
            )
            sample_name = f"Sample_{exp_id}"
            energy_start = 250.0
            energy_stop = 280.0
        else:
            df = template_data["df"].copy()
            sample_name = template_data["sample_name"]
            energy_start = template_data["energy_start"]
            energy_stop = template_data["energy_stop"]

        # Individual experiment widgets
        exp_name = pn.widgets.TextInput(
            name="Sample Name", value=sample_name, width=300
        )

        exp_energy_start = pn.widgets.FloatInput(
            name="Energy Start [eV]", value=energy_start, width=300
        )

        exp_energy_stop = pn.widgets.FloatInput(
            name="Energy Stop [eV]", value=energy_stop, width=300
        )

        exp_table = pn.widgets.Tabulator(
            df,
            width=750,
            height=300,
            layout="fit_columns",
            editors={
                "Step": None,
                "Sample Theta": {"type": "number"},
                "Higher Order Suppressor": {"type": "number"},
                "Exposure": {"type": "number"},
            },
        )

        # Row controls for this experiment
        def add_row_to_exp(event):
            current = exp_table.value
            new_step = len(current) + 1
            new_row = {
                "Step": new_step,
                "Sample Theta": 0.0,
                "Higher Order Suppressor": 10.0,
                "Exposure": 0.001,
            }
            exp_table.value = pd.concat(
                [current, pd.DataFrame([new_row])], ignore_index=True
            )

        def remove_row_from_exp(event):
            current = exp_table.value
            if len(current) > 1:
                exp_table.value = current.iloc[:-1].reset_index(drop=True)

        add_row_btn = pn.widgets.Button(
            name="Add Row", button_type="primary", width=120
        )
        add_row_btn.on_click(add_row_to_exp)

        remove_row_btn = pn.widgets.Button(
            name="Remove Row", button_type="danger", width=120
        )
        remove_row_btn.on_click(remove_row_from_exp)

        # Build experiment panel
        exp_panel = pn.Column(
            pn.pane.Markdown(
                f"### Experiment {exp_id}",
                styles={"background": "#e8f4f8", "padding": "10px"},
            ),
            pn.Row(exp_name, exp_energy_start, exp_energy_stop),
            pn.pane.Markdown("#### Measurement Steps"),
            exp_table,
            pn.Row(add_row_btn, remove_row_btn),
            styles={"padding": "10px"},
        )

        # Store widgets for later access
        self.experiments[exp_id] = {
            "panel": exp_panel,
            "name": exp_name,
            "energy_start": exp_energy_start,
            "energy_stop": exp_energy_stop,
            "table": exp_table,
        }

        return exp_panel

    def add_experiment(self, event):
        """Add a new experiment tab"""
        self.experiment_count += 1
        exp_id = self.experiment_count

        exp_panel = self.create_experiment_panel(exp_id)
        self.tabs.append((f"Experiment {exp_id}", exp_panel))

    def remove_experiment(self, event):
        """Remove the currently active experiment tab"""
        if len(self.tabs) > 1:
            active_index = self.tabs.active
            # Remove from tabs
            self.tabs.pop(active_index)
            # Clean up from experiments dict
            exp_to_remove = None
            for exp_id, exp_data in self.experiments.items():
                if self.tabs[active_index] == exp_data["panel"]:
                    exp_to_remove = exp_id
                    break
            if exp_to_remove:
                del self.experiments[exp_to_remove]

    def duplicate_experiment(self, event):
        """Duplicate the currently active experiment"""
        if len(self.tabs) > 0:
            active_index = self.tabs.active
            # Find the active experiment
            for exp_id, exp_data in self.experiments.items():
                if (
                    active_index < len(self.tabs)
                    and self.tabs[active_index][1] == exp_data["panel"]
                ):
                    # Extract current data
                    template_data = {
                        "df": exp_data["table"].value,
                        "sample_name": exp_data["name"].value + "_copy",
                        "energy_start": exp_data["energy_start"].value,
                        "energy_stop": exp_data["energy_stop"].value,
                    }
                    # Create new experiment with template
                    self.experiment_count += 1
                    new_exp_id = self.experiment_count
                    new_panel = self.create_experiment_panel(new_exp_id, template_data)
                    self.tabs.append((f"Experiment {new_exp_id}", new_panel))
                    break

    def export_all(self):
        """Export all experiments to a summary"""
        summary_text = f"# Multi-Experiment Script: {self.script_name.value}\n\n"

        for exp_id, exp_data in self.experiments.items():
            summary_text += f"## Experiment {exp_id}: {exp_data['name'].value}\n"
            summary_text += f"- Energy Range: {exp_data['energy_start'].value} - {exp_data['energy_stop'].value} eV\n"
            summary_text += f"- Number of Steps: {len(exp_data['table'].value)}\n\n"

        return summary_text

    def build_layout(self):
        """Build the complete multi-experiment interface"""

        # Header
        header = pn.Column(
            pn.pane.Markdown(
                "# Multi-Experiment Builder",
                styles={"background": "#6a1b9a", "color": "white", "padding": "20px"},
            ),
            pn.Row(
                self.script_name,
                pn.pane.Markdown(
                    "*Manage multiple experiments with tabs*",
                    styles={"padding": "10px"},
                ),
            ),
        )

        # Tab controls
        tab_controls = pn.Row(
            self.add_experiment_button,
            self.duplicate_experiment_button,
            self.remove_experiment_button,
            styles={"background": "#f0f0f0", "padding": "10px"},
        )

        # Main app
        app = pn.Column(
            header, tab_controls, self.tabs, width=900, styles={"background": "white"}
        )

        return app


# Create and display the multi-experiment builder
multi_builder = MultiExperimentBuilder()
multi_app = multi_builder.build_layout()
multi_app

BokehModel(combine_events=True, render_bundle={'docs_json': {'3c247c6a-411b-43b9-9d2e-b259320de87d': {'version…

## 4. Accordion Menus for Organized Parameters

Panel provides Accordion widgets similar to ipywidgets, great for organizing many parameters:


In [ ]:
# Create parameter groups using Accordion (similar to ipywidgets Accordion)

# Motor Positions Group
motor_params = pn.Column(
    pn.widgets.FloatInput(name="Sample X [mm]", value=0.0, width=250),
    pn.widgets.FloatInput(name="Sample Y [mm]", value=0.0, width=250),
    pn.widgets.FloatInput(name="Sample Z [mm]", value=0.0, width=250),
    pn.widgets.FloatInput(name="Sample Z Flip [mm]", value=0.0, width=250),
    pn.widgets.FloatInput(name="Direct Beam Z [mm]", value=-5.0, width=250),
)

# Optional Offsets Group
offset_params = pn.Column(
    pn.widgets.FloatInput(name="X Offset [mm]", value=0.15, width=250),
    pn.widgets.FloatInput(name="Theta Offset [deg]", value=0.0, width=250),
    pn.widgets.Checkbox(name="Reverse Holder (Start at -180°)", value=False, width=250),
)

# Advanced Options Group
advanced_params = pn.Column(
    pn.widgets.Select(
        name="Independent Variable", options=["Qval", "Theta"], value="Qval", width=250
    ),
    pn.widgets.Select(
        name="Point Density Method", options=["Use Thickness", "Use Delta"], width=250
    ),
    pn.widgets.FloatInput(name="Sample Thickness [Å]", value=250, width=250),
    pn.widgets.TextInput(
        name="Angle Crossover", value="0, 10", placeholder="comma-separated", width=250
    ),
    pn.widgets.TextInput(
        name="Point Density", value="15, 6", placeholder="comma-separated", width=250
    ),
    pn.widgets.IntInput(name="Overlap Points", value=4, width=250),
    pn.widgets.IntInput(name="Buffer Points", value=2, width=250),
    pn.widgets.IntInput(name="I0 Points", value=10, width=250),
)

# Create Accordion
accordion = pn.Accordion(
    ("Motor Positions", motor_params),
    ("Optional Offsets", offset_params),
    ("Advanced Options", advanced_params),
    width=600,
    active=[0],  # First section open by default
)

# Display with a header
pn.Column(
    pn.pane.Markdown(
        "### Organized Parameters with Accordion",
        styles={"background": "#4CAF50", "color": "white", "padding": "10px"},
    ),
    accordion,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e6d7ebf0-e923-484e-b0e2-03c583e0dfdd': {'version…

## 5. Reactive Binding & Live Updates

One of Panel's most powerful features is reactive binding - widgets automatically update based on other widget values:


In [ ]:
# Reactive DataFrame example - table updates automatically when parameters change

# Input parameters
num_rows = pn.widgets.IntSlider(
    name="Number of Rows", start=1, end=20, value=5, width=300
)
theta_start = pn.widgets.FloatInput(name="Starting Theta", value=1.0, width=200)
theta_step = pn.widgets.FloatInput(name="Theta Step", value=5.0, width=200)
exposure_base = pn.widgets.FloatInput(name="Base Exposure", value=0.001, width=200)


# Reactive function that generates DataFrame based on widget values
@pn.depends(
    num_rows.param.value,
    theta_start.param.value,
    theta_step.param.value,
    exposure_base.param.value,
)
def generate_reactive_table(n_rows, theta_start_val, theta_step_val, exposure_val):
    """Generate table that updates automatically when inputs change"""

    thetas = [theta_start_val + i * theta_step_val for i in range(n_rows)]
    exposures = [exposure_val * (1 + i * 0.1) for i in range(n_rows)]
    hos = [10.0 - i * 0.2 for i in range(n_rows)]

    df = pd.DataFrame(
        {
            "Step": range(1, n_rows + 1),
            "Sample Theta": thetas,
            "Higher Order Suppressor": hos,
            "Exposure": exposures,
        }
    )

    return pn.widgets.Tabulator(df, width=600, height=350, layout="fit_columns")


# Summary statistics
@pn.depends(
    num_rows.param.value,
    theta_start.param.value,
    theta_step.param.value,
    exposure_base.param.value,
)
def generate_summary(n_rows, theta_start_val, theta_step_val, exposure_val):
    """Generate summary statistics"""
    theta_end = theta_start_val + (n_rows - 1) * theta_step_val
    total_exposure = sum([exposure_val * (1 + i * 0.1) for i in range(n_rows)])

    summary = f"""
    **Table Summary:**
    - Total Steps: {n_rows}
    - Theta Range: {theta_start_val:.2f}° to {theta_end:.2f}°
    - Total Exposure Time: {total_exposure:.4f} s
    """
    return pn.pane.Markdown(
        summary, styles={"background": "#e3f2fd", "padding": "10px"}
    )


# Layout
reactive_app = pn.Column(
    pn.pane.Markdown(
        "### Reactive Table Generator",
        styles={"background": "#FF9800", "color": "white", "padding": "10px"},
    ),
    pn.pane.Markdown(
        "*Change parameters and watch the table update automatically!*",
        styles={"font-style": "italic", "padding": "5px"},
    ),
    pn.Row(
        pn.Column(num_rows, theta_start, theta_step, exposure_base), generate_summary
    ),
    generate_reactive_table,
    width=800,
)

reactive_app

BokehModel(combine_events=True, render_bundle={'docs_json': {'6cef2cb8-ae81-4676-a992-da9753ecfe70': {'version…

## 6. File Browser & Save Dialog

Panel can integrate file selection and save dialogs similar to your PyQt6 implementation:


In [ ]:
# File Download capabilities in Panel

# Sample data
sample_df = pd.DataFrame(
    {
        "Sample": ["Sample_A", "Sample_B", "Sample_C"],
        "Energy_Start": [250, 260, 270],
        "Energy_Stop": [280, 290, 300],
        "Num_Steps": [10, 15, 12],
    }
)

# TextInput for filename
filename_input = pn.widgets.TextInput(
    name="Filename", value="experiment_data", width=300
)


# Function to create CSV download
def get_csv_download():
    """Create a file download widget with current data"""
    sio = StringIO()
    sample_df.to_csv(sio, index=False)
    sio.seek(0)

    filename = filename_input.value
    if not filename.endswith(".csv"):
        filename += ".csv"

    return pn.widgets.FileDownload(
        sio,
        filename=filename,
        button_type="success",
        label=f"Download {filename}",
        width=200,
    )


# Status message
status_msg = pn.pane.Markdown("", width=400)


def update_status(event):
    """Update status when download is clicked"""
    status_msg.object = f"✓ File prepared: **{filename_input.value}.csv**"


# Create download button with binding
download_button = pn.bind(get_csv_download)

# Layout
file_app = pn.Column(
    pn.pane.Markdown(
        "### File Download Demo",
        styles={"background": "#00796B", "color": "white", "padding": "10px"},
    ),
    pn.widgets.Tabulator(sample_df, width=500, height=200),
    pn.Row(filename_input, download_button),
    status_msg,
    width=600,
)

file_app

BokehModel(combine_events=True, render_bundle={'docs_json': {'08ae6b8e-b5e9-4386-8ccf-d227946f9ea0': {'version…

## 7. JSON Load/Save Functionality

Similar to your JSON save/load in the ipywidgets version:


In [ ]:
import json


class JSONConfigBuilder:
    """
    Demonstrates JSON-based save/load functionality
    """

    def __init__(self):
        # Parameters
        self.sample_name = pn.widgets.TextInput(
            name="Sample Name", value="MySample", width=250
        )
        self.energy_start = pn.widgets.FloatInput(
            name="Energy Start", value=250.0, width=250
        )
        self.energy_stop = pn.widgets.FloatInput(
            name="Energy Stop", value=280.0, width=250
        )
        self.x_position = pn.widgets.FloatInput(name="X Position", value=0.0, width=250)
        self.y_position = pn.widgets.FloatInput(name="Y Position", value=0.0, width=250)

        # Table
        self.df = pd.DataFrame(
            {
                "Step": [1, 2, 3],
                "Theta": [1.0, 5.0, 10.0],
                "Exposure": [0.001, 0.001, 0.01],
            }
        )

        self.table = pn.widgets.Tabulator(self.df, width=500, height=250)

        # Status
        self.status = pn.pane.Markdown("", width=600)

        # Config name
        self.config_name = pn.widgets.TextInput(
            name="Config Name", value="my_config", width=250
        )

    def get_config_dict(self):
        """Extract all parameters as a dictionary"""
        config = {
            "sample_name": self.sample_name.value,
            "energy_start": self.energy_start.value,
            "energy_stop": self.energy_stop.value,
            "x_position": self.x_position.value,
            "y_position": self.y_position.value,
            "table_data": self.table.value.to_dict("records"),
        }
        return config

    def load_config_dict(self, config):
        """Load parameters from a dictionary"""
        self.sample_name.value = config.get("sample_name", "")
        self.energy_start.value = config.get("energy_start", 0.0)
        self.energy_stop.value = config.get("energy_stop", 0.0)
        self.x_position.value = config.get("x_position", 0.0)
        self.y_position.value = config.get("y_position", 0.0)

        if "table_data" in config:
            self.table.value = pd.DataFrame(config["table_data"])

    def save_to_json(self, event):
        """Save current configuration to JSON file"""
        config = self.get_config_dict()
        filename = self.config_name.value
        if not filename.endswith(".json"):
            filename += ".json"

        with open(filename, "w") as f:
            json.dump(config, f, indent=2)

        self.status.object = f"✓ Configuration saved to **{filename}**"

    def load_from_json(self, event):
        """Load configuration from JSON file"""
        filename = self.config_name.value
        if not filename.endswith(".json"):
            filename += ".json"

        try:
            with open(filename, "r") as f:
                config = json.load(f)

            self.load_config_dict(config)
            self.status.object = f"✓ Configuration loaded from **{filename}**"
        except FileNotFoundError:
            self.status.object = f"❌ File **{filename}** not found"
        except json.JSONDecodeError:
            self.status.object = f"❌ Error reading JSON from **{filename}**"

    def get_json_preview(self, event):
        """Show JSON preview"""
        config = self.get_config_dict()
        json_str = json.dumps(config, indent=2)
        self.status.object = f"**JSON Preview:**\n```json\n{json_str}\n```"

    def build_layout(self):
        """Build the interface"""

        # Save/Load buttons
        save_button = pn.widgets.Button(
            name="Save JSON", button_type="success", width=120
        )
        save_button.on_click(self.save_to_json)

        load_button = pn.widgets.Button(
            name="Load JSON", button_type="primary", width=120
        )
        load_button.on_click(self.load_from_json)

        preview_button = pn.widgets.Button(
            name="Preview JSON", button_type="warning", width=120
        )
        preview_button.on_click(self.get_json_preview)

        app = pn.Column(
            pn.pane.Markdown(
                "### JSON Configuration Manager",
                styles={"background": "#512DA8", "color": "white", "padding": "10px"},
            ),
            pn.pane.Markdown("*Save and load your configurations as JSON files*"),
            pn.Row(self.sample_name, self.energy_start),
            pn.Row(self.energy_stop, self.x_position),
            self.y_position,
            pn.pane.Markdown("#### Measurement Table"),
            self.table,
            pn.pane.Markdown("#### Save/Load Controls"),
            pn.Row(self.config_name, save_button, load_button, preview_button),
            self.status,
            width=650,
        )

        return app


# Create and display
json_builder = JSONConfigBuilder()
json_app = json_builder.build_layout()
json_app

BokehModel(combine_events=True, render_bundle={'docs_json': {'f38879a0-7a89-400f-b13a-9b879790d9bf': {'version…

## Summary: Panel vs ipywidgets

### Key Advantages of Panel:

1. **More Modern & Responsive**: Better styling options and responsive layouts
2. **Tabulator Widget**: More powerful than basic DataFrame display with built-in editing
3. **Reactive Programming**: `@pn.depends` decorator makes it easy to create auto-updating components
4. **Better Layouts**: FlexBox-based layouts, Accordion, Tabs work seamlessly
5. **Deployment Ready**: Easy to convert notebooks to standalone web apps with `panel serve`
6. **Integration**: Works with Bokeh, HoloViews, Plotly, Matplotlib, etc.

### Features Demonstrated:

- ✅ Editable tables with Tabulator
- ✅ Dynamic add/remove rows and columns
- ✅ Global parameters with various input types
- ✅ Tab-based multi-experiment interface
- ✅ Accordion for organized parameters
- ✅ Reactive binding for automatic updates
- ✅ File download functionality
- ✅ JSON save/load
- ✅ Custom styling and theming

### To Deploy as Web App:

```bash
panel serve panel.ipynb --show
```

This creates a standalone web application accessible through a browser!


# Full PXR Script Generator Implementation

Now let's use the Panel-based implementation of the BaseWidgets and PXR_Widgets modules to create the exact same functionality as the ipywidgets version.


In [ ]:
# Import the Panel-based widget modules
from Panel_PXR_Widgets import PXR_Experiment, PXR_Scan, PXR_ScriptGen

# Initialize Panel extension if not already done
pn.extension("tabulator")

# Create the PXR Script Generator
# This creates the full hierarchical widget structure:
# ScriptGen -> Multiple Experiments (samples) -> Multiple Scans (measurements) -> Motor positions
pxr_generator = PXR_ScriptGen(path=".")

# Display the full interface
pxr_generator.GUI

## Understanding the Widget Hierarchy

The PXR Script Generator has a three-level hierarchy similar to the ipywidgets version:

### Level 1: PXR_ScriptGen (Top Level)

- Manages multiple sample experiments
- Provides save/load JSON functionality
- Controls for adding/removing/duplicating samples
- Generates final beamline scripts

### Level 2: PXR_Experiment (Sample Level)

- Manages motor positions and global parameters for one sample
- Contains Accordion menus for organized parameter groups:
  - Motor Positions (X, Y, Z coordinates)
  - Optional Offsets
  - Advanced Options (point density, buffers, etc.)
- Manages multiple measurement scans

### Level 3: PXR_Scan (Measurement Level)

- Defines energy range and scan parameters
- Dynamic table for motor positions per step
- Controls for adding/removing steps and motors
- Each row represents a motor, each column a step

### Key Features Implemented:

1. **Dynamic Tables**: Add/remove rows (motors) and columns (steps)
2. **Nested Tabs**: Samples contain measurement tabs
3. **Parameter Organization**: Accordion menus for complex parameters
4. **Save/Load**: JSON configuration persistence
5. **Script Export**: Generate ALS beamline-compatible scripts


## Usage Examples

Here are some common operations you can perform:


In [ ]:
# Example 1: Access current configuration
config = pxr_generator.save_as_dict()
print("Current configuration keys:", config.keys())

# Example 2: Programmatically add a new sample
# This simulates clicking the "Add New Sample" button
pxr_generator.new_tab(pxr_generator.layout, PXR_Experiment, PXR_Experiment.ALS_NAME)

# Example 3: Set save directory
pxr_generator.save_dir = "/home/hduva/projects/xrr_notebooks/data"
print(f"Save directory set to: {pxr_generator.save_dir}")

# Example 4: Set script name
pxr_generator.save_name.value = "my_pxr_experiment"
print(f"Script will be saved as: {pxr_generator.save_name.value}")

# Example 5: Access a specific experiment
# Get first experiment (Sample 1)
exp_key = f"{PXR_Experiment.ALS_NAME}{pad_digits(1)}"
if hasattr(pxr_generator, exp_key):
    first_exp = getattr(pxr_generator, exp_key)
    print(f"First experiment sample name: {first_exp.name_of_sample.value}")

    # Modify a parameter
    first_exp.XPosition.value = 5.0
    print(f"Updated X Position to: {first_exp.XPosition.value}")

## Comparison: Panel vs ipywidgets Implementation

| Feature            | ipywidgets Version         | Panel Version                |
| ------------------ | -------------------------- | ---------------------------- |
| **UI Framework**   | Jupyter widgets            | HoloViz Panel                |
| **Table Widget**   | Manual HTML/VBox layout    | Tabulator (built-in editing) |
| **Deployment**     | Jupyter only               | Jupyter + standalone web app |
| **File Browser**   | PyQt6 dialogs              | Text input (or custom)       |
| **Styling**        | CSS via style dicts        | Modern CSS + themes          |
| **Reactivity**     | Manual callbacks           | `@pn.depends` decorators     |
| **Layouts**        | HBox, VBox, Tab, Accordion | Row, Column, Tabs, Accordion |
| **Responsiveness** | Fixed sizes                | Responsive by default        |

### Advantages of Panel Version:

1. **Better Table Editing**: Tabulator widget is more powerful than custom HTML tables
2. **Web Deployment**: Can run as standalone app with `panel serve`
3. **Modern UI**: Better default styling and responsive layouts
4. **Easier Maintenance**: Less custom layout code needed
5. **Integration**: Works with Bokeh, HoloViews, Plotly for visualizations

### Migration Notes:

- All core functionality preserved
- Widget hierarchy identical
- Configuration structure compatible
- Some UI interactions simplified (file browser uses text input instead of PyQt6 dialogs)


## Next Steps

### To use this in production:

1. **Install Panel**:

   ```bash
   pip install panel
   ```

2. **Run in Jupyter**:

   - Simply execute the cells above
   - Interactive widgets work directly in notebooks

3. **Deploy as Web App**:

   ```bash
   panel serve panel.ipynb --show
   ```

   - Creates standalone web application
   - Accessible via browser at http://localhost:5006
   - Can be deployed to servers

4. **Convert to Python script**:
   - Extract the class definitions into a `.py` file
   - Import and use: `from Panel_PXR_Widgets import PXR_ScriptGen`

### Files Created:

- **Panel_BaseWidgets.py**: Base classes for Panel-based ALS widgets
- **Panel_PXR_Widgets.py**: PXR-specific implementation using Panel
- **panel.ipynb**: This demonstration notebook

### Customization:

You can extend these classes to add:

- Additional motor types
- Custom validation logic
- Data visualization with Bokeh/HoloViews
- Real-time preview of scan parameters
- Integration with beamline control systems
